In [1]:
import sys
sys.path.append('..')
import atclient
import pandas as pd

## Use the API

### Connect

Create a config.yaml with 3 fields : url, username and password

In [2]:
api = atclient.AtApi(config='../config.yaml')


Token received


### Projects management

Load data

In [3]:
df = pd.read_parquet("../../activetigger/data/dataset_test.parquet")
df.head()

,Unnamed: 0,id,text,label,prob,proba
0,0,1569866966566932480,"Les ENFANTS du Québec, \nLes AÎNÉS du Québec,\...",Autre,0.823412,NaN
1,1,1569919081351942144,Après avoir imposé une politique dystopique zé...,Autre,0.599684,NaN
2,2,1569930968026480640,"@Nyonensis @DIVIZIO1 Pour le coup, si ce qu'il...",Autre,0.941126,NaN
3,3,1569947515465728000,@nadinecerdan @SaiyanBio @nawah_s C'est sous e...,Autre,0.625537,NaN
4,4,1569961852972732416,@L_ThinkTank L'achat de vaccins et d'armes ne ...,Autre,0.103508,NaN


Get available projects

In [3]:
r = api.get_projects()
len(r)

3

Get project slugs

In [4]:
api.get_projects_slugs()


['off-the-record-adam', 'off-the-record-imane', 'featurization-edito-pol']

(If you want to sluggify a project name, you can use the following function)

```python
from slugify import slugify
slugify("My project name")
```

Create a project

In [4]:
api.add_project("test3", df, "id", ["text"], col_label="label")

b'"test3"'


Delete a project

In [4]:
api.delete_project("test3")

b'{"detail":"Project doesn\'t exist"}'


### Users management

Add

In [5]:
api.add_user("coucou2", "coucou2", "test@test.fr")

User created


Delete

In [12]:
api.delete_user("coucou2")

User deleted


Add Auth for a project

In [6]:
api.add_auth_user_project("coucou2", "test3", "manager")

Auth added to user


Delete Auth for a project

In [3]:
api.delete_auth_user_project("coucou2", "test3")

Auth deleted for user


### Get project state

In [6]:
state = api.get_project_state("test")
state["schemes"]

{'available': {'default': {'labels': ['Autre', 'Règles'],
   'kind': 'multiclass'},
  'coucou': {'labels': ['Super', 'Sous'], 'kind': 'multiclass'}},
 'statistics': {}}

### Get current annotations

In [15]:
t = api.get_annotations_data("test", "default")
t.head()

,Unnamed: 0,id,dataset_id,text,labels,user,timestamp,comment
0,0,1569866966566932480,1569866966566932480,"Les ENFANTS du Québec, \nLes AÎNÉS du Québec,\...",Autre,root,2025-01-08 17:08:11.010291,NaN
1,1,1569919081351942144,1569919081351942144,Après avoir imposé une politique dystopique zé...,Autre,root,2025-01-08 17:08:11.010386,NaN
2,2,1569930968026480640,1569930968026480640,"@Nyonensis @DIVIZIO1 Pour le coup, si ce qu'il...",Autre,root,2025-01-08 17:08:11.010416,NaN
3,3,1569947515465728000,1569947515465728000,@nadinecerdan @SaiyanBio @nawah_s C'est sous e...,Autre,root,2025-01-08 17:08:11.010438,NaN
4,4,1569961852972732416,1569961852972732416,@L_ThinkTank L'achat de vaccins et d'armes ne ...,Autre,root,2025-01-08 17:08:11.010459,NaN


### Manage features

Get available feature

In [7]:
r = api.get_features("test3")
r.keys()

dict_keys(['sbert'])

Add a SBERT feature on the train dataset

In [5]:
api.add_feature("test3", "sbert", "sbert")

b'{"detail":"computing sbert, it could take a few minutes","status":"waiting"}'


Get features

In [4]:
df = api.get_features_data("test3", ["sbert"])
df.head()

,id,sbert__sb001,sbert__sb002,sbert__sb003,sbert__sb004,sbert__sb005,sbert__sb006,sbert__sb007,sbert__sb008,sbert__sb009,...,sbert__sb503,sbert__sb504,sbert__sb505,sbert__sb506,sbert__sb507,sbert__sb508,sbert__sb509,sbert__sb510,sbert__sb511,sbert__sb512
0,1569866966566932480,-0.011765,-0.039000,0.022600,-0.036620,0.05145,-0.040440,-0.023510,-0.00535,0.01270,...,-0.06800,-0.02504,-0.040700,-0.001096,-0.06866,0.052500,-0.009926,0.01254,-0.03260,-0.056430
1,1569919081351942144,0.014670,-0.016880,0.002716,-0.033500,0.02348,-0.048300,0.060060,0.08030,-0.02367,...,-0.07250,-0.01132,0.054350,0.100500,-0.02869,0.005985,0.008736,-0.00974,-0.03983,-0.063660
2,1569930968026480640,0.021730,-0.011480,0.034970,-0.042400,0.06370,-0.026170,0.042940,0.02400,-0.02003,...,0.01248,0.02101,-0.002077,-0.014250,-0.03348,0.047300,-0.001765,-0.09290,-0.03075,-0.015300
3,1569947515465728000,0.013320,0.002968,-0.027510,-0.065250,0.05127,-0.085400,0.014694,0.05295,-0.04828,...,-0.05408,-0.00360,0.005302,0.009766,-0.02873,0.006115,-0.016530,-0.02817,-0.01456,-0.029330
4,1569961852972732416,0.065730,0.019930,0.035220,0.011986,-0.04720,-0.007717,-0.009430,0.03375,-0.07680,...,-0.07330,0.02222,0.026210,0.038120,-0.10016,0.020020,0.029590,-0.09950,-0.00475,-0.014435


## Manage scheme

Get available scheme

In [7]:
api.get_schemes("genre")

{'default': {'labels': ['pas_genre', 'genre'], 'kind': 'multiclass'}}

Add scheme

In [4]:
api.add_scheme_to_project("genre","test",["coucou","essai"])

Scheme added to project


Delete scheme

In [6]:
api.delete_scheme_from_project("genre","test")

Scheme deleted from project


Add a label to a scheme

In [4]:
api.add_label_to_scheme("genre", "default","test")

Label added to scheme


Remove label from a scheme

In [5]:
api.delete_label_from_scheme("genre", "default", "test")

Label deleted from scheme
